In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing important packages
import cudf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
from sklearn.model_selection import GridSearchCV  #Perforing grid search
import json
from sklearn.metrics import roc_auc_score
from time import process_time

In [ ]:
!mkdir result

In [ ]:
params = {
#     'max_depth': 5,
#     'eta': 0.1,
#     'min_child_weight': 35,
#     'subsample': 0.1
     'tree_method' : 'gpu_hist'
}

#max depth 15
#min_child_weight 20
#subsample 0.9
#eta 0.1


#{'eta': 0.1, 'max_depth': 15, 'min_child_weight': 30, 'subsample': 0.9}

#{'eta': 0.1, 'max_depth': 5, 'min_child_weight': 35, 'subsample': 0.1} for multil model

In [ ]:
df_test = pd.read_csv('/kaggle/input/train-test-valid/merge_hdfs_item_valid.csv',header=None,index_col=False)

In [ ]:
df_test.head()

In [ ]:
list_train = []
list_test = []
list_time = []
for i in range(4,10):
    df_train = pd.read_csv('/kaggle/input/final-features/item_train_' + str(i) + '.csv',index_col=False)
    
    x_train = df_train.drop('1',axis=1)
    y_train = df_train['1']
    del df_train
    
    t1_start = process_time() 

    
    model = xgb.XGBClassifier(**params)
    model.fit(x_train,y_train)
    
    t1_stop = process_time()
    
    predict_train = model.predict_proba(x_train)[:, 1]

    # Accuray Score on train dataset
    accuracy_train = roc_auc_score(y_train,predict_train)
    print('\naccuracy_score on train dataset : {:.2f}%'.format(accuracy_train*100))
    list_train.append(accuracy_train*100)
    del x_train
    del y_train
    
    x_test = df_test.drop(1,axis=1)
    y_test = df_test[1]
    
    predict_test = model.predict_proba(x_test)[:, 1]

    # Accuracy Score on test dataset
    accuracy_test = roc_auc_score(y_test,predict_test)
    print('\naccuracy_score on test dataset : {:.2f}%'.format(accuracy_test*100))
    list_test.append(accuracy_test*100)
    
    filename = '/kaggle/working/result/xgboost_model_'+ str(i)+'.sav'
    pickle.dump(model, open(filename, 'wb'))
    
    time_train = t1_stop - t1_start
    list_time.append(time_train)
    
    f = open("/kaggle/working/result/xgboost_params_test.txt", "a")
    f.write('\nmodel_'+ str(i) + ": " + json.dumps(params) + "\t" + "train: " + str(accuracy_train*100) + "\t" + "test: "+  str(accuracy_test*100)+ "\ttime: "+ str(time_train)+"\n")
    f.close()
del x_test
del y_test

In [ ]:
import statistics

f = open("/kaggle/working/result/xgboost_params_test.txt", "a")
f.write("\naverage naccuracy_score on train dataset: %f" % statistics.mean(list_train))
f.write("\naverage naccuracy_score on test dataset: %f\n" % statistics.mean(list_test))
f.write("\naverage training time: %f\n" % statistics.mean(list_time))
f.close()
print("\naverage naccuracy_score on train dataset: %f" % statistics.mean(list_train))
print("\naverage naccuracy_score on test dataset: %f" % statistics.mean(list_test))
print("\naverage training time: %f\n" % statistics.mean(list_time))

In [ ]:
#!pwd

In [ ]:
#df_test.dropna(inplace=True)

In [ ]:
# categorical_feature_mask = df_test.dtypes==object
# # Get list of categorical column names
# categorical_cols = df_test.columns[categorical_feature_mask].tolist()
# # apply le on categorical feature columns
# df_test[categorical_cols] = df_test[categorical_cols].apply(lambda col: le.fit_transform(col))

In [ ]:
# x_test = df_test.drop('1',axis=1)
# y_test = df_test['1']
# del df_test

In [ ]:
# y_pred = model.predict(x_test)
# del x_test

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,)

In [ ]:
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_pred, y_test)
# del y_test
# print(accuracy)